In [ ]:
# Use seaborn for pairplot
!pip install -q seaborn

# Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import IPython

In [ ]:
import tensorflow as tf
!pip install -q -U keras-tuner
import kerastuner as kt
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:

train_dataset = pd.read_csv('')
test_dataset = pd.read_csv('')

In [ ]:
train_labels = train_dataset.pop("sboxes")
test_labels = test_dataset.pop("sboxes")

In [ ]:
from keras import backend as K
from tensorflow.python.framework import ops
from tensorflow.python.ops import math_ops
def root_mean_squared_error(y_true, y_pred):
        y_pred = ops.convert_to_tensor_v2(y_pred)
        y_true = math_ops.cast(y_true, y_pred.dtype)
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [ ]:
def R_squared(y, y_pred):
  y_pred = ops.convert_to_tensor_v2(y_pred)
  y = math_ops.cast(y, y_pred.dtype)  
  residual = tf.reduce_sum(tf.square(tf.subtract(y,y_pred)))
  total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
  r2 = tf.subtract(1.0, tf.math.divide(residual, total))
  return r2

In [ ]:
def quantile(self, y_true, y_pred):
        logits_pred = y_pred
        theta_target = y_true
        
        theta_pred = tf.reduce_sum(tf.multiply(self.action_binary_loss , logits_pred), axis=1)
        
        theta_target_tile = tf.tile(tf.expand_dims(theta_target, axis=2), [1, 1, self.n_quantiles])
        theta_pred_tile = tf.tile(tf.expand_dims(theta_pred, axis=1), [1, self.n_quantiles, 1])
        
        error_loss = self.theta_target_tile - self.theta_pred_tile        
        huber_loss = self.HuberLoss(theta_target_tile, theta_pred_tile)
       
        min_tau = 1 / (2 * self.n_quantiles)
        max_tau = (2*self.n_quantiles +1) / (2*self.n_quantiles)
        
        tau = tf.reshape(tf.range(min_tau, max_tau, 1/self.n_quantiles), [1, self.n_quantiles])
        inv_tau = 1.0 - tau
        
        Loss = tf.where(tf.less(error_loss, 0.0), inv_tau * huber_loss, tau* huber_loss)
        Loss = tf.reduce_mean(tf.reduce_sum(tf.reduce_mean(Loss, axis=2), axis=1))
        
        return Loss

In [ ]:
def above_percentile(x, p): #assuming the input is flattened: (n,)

    samples = K.cast(K.shape(x)[0], K.floatx()) #batch size
    p =  (100. - p)/100.  #100% will return 0 elements, 0% will return all elements

    #samples to get:
    samples = K.cast(tf.math.floor(p * samples), 'int32')
        #you can choose tf.math.ceil above, it depends on whether you want to
        #include or exclude one element. Suppose you you want 33% top,
        #but it's only possible to get exactly 30% or 40% top:
        #floor will get 30% top and ceil will get 40% top.
        #(exact matches included in both cases)

    #selected samples
    values, indices = tf.math.top_k(x, samples)

    return values

def custom_loss(p):
    def loss(y_true, y_predicted):
        y_predicted = ops.convert_to_tensor_v2(y_predicted)
        y_true = math_ops.cast(y_true, y_predicted.dtype)  
        ses = K.square(y_true-y_predicted)
        above = above_percentile(K.flatten(ses), p)
        return K.mean(above)
    return loss

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='elu', input_shape=[len(train_dataset.keys())], kernel_initializer='he_uniform'),
    layers.Dense(64, activation='elu'),
    layers.Dense(64, activation='elu'),
    layers.Dense(64, activation='elu'),
    layers.Dense(64, activation='elu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
  model.compile(loss=root_mean_squared_error,
                optimizer=optimizer,
                metrics=['mae',root_mean_squared_error,R_squared])
  return model

In [ ]:
model = build_model()


In [ ]:
model.summary()

In [ ]:
EPOCHS =200

history = model.fit(
  train_dataset, train_labels, shuffle=True, 
  epochs=EPOCHS, batch_size=32, validation_batch_size=None, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])
IPython.display.clear_output(wait = True)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)

In [ ]:
plotter.plot({'Basic': history}, metric = "mae")
plt.ylim([0, 10])
plt.ylabel('MAE [ActiveSboxes]')

In [ ]:
loss,mae,root_mean_squared_error,R_squared= model.evaluate(test_dataset, test_labels,batch_size= None,
   verbose=2,
  callbacks=[tfdocs.modeling.EpochDots()])

print("Testing set Mean Abs Error: {:5.2f} ActiveSboxes".format(mae))
print("Testing set Root Mean  Square Error: {:5.2f} ActiveSboxes".format(root_mean_squared_error))
print("Testing set R Square: {:5.2f} ActiveSboxes".format(R_squared))

In [ ]:

test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [ActiveSboxes]')
plt.ylabel('Predictions [ActiveSboxes]')
lims = [0, 60]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)